In [1]:
# !pip install pytorch-tabnet

In [5]:
import os
from glob import glob
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.metrics import Metric

import torch.nn.functional as F
import torch.optim as optim
# from datasets import Dataset
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler # 표준화 패키지 라이브러리
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import pickle
from tqdm import tqdm
import random
from warnings import filterwarnings
import gc
from collections import Counter
import matplotlib.pyplot as plt

filterwarnings('ignore')
device = torch.device('cpu')
os.getcwd()

'/Users/louan/Documents/projects/Aiffel/Hackathon/geodata-con/geo-con/model/tabnet'

# Data Load

In [14]:
path = '../../../data/*.pkl'

file_lst = sorted(glob(path))[1:]
file_lst[6] = '../../../data/pre_processed_split_0tpt_data_6.pkl'
print(file_lst)

['../../../data/pre_processed_split_data_0.pkl', '../../../data/pre_processed_split_data_1.pkl', '../../../data/pre_processed_split_data_2.pkl', '../../../data/pre_processed_split_data_3.pkl', '../../../data/pre_processed_split_data_4.pkl', '../../../data/pre_processed_split_data_5.pkl', '../../../data/pre_processed_split_0tpt_data_6.pkl', '../../../data/pre_processed_split_data_7.pkl', '../../../data/pre_processed_split_data_8.pkl']


In [19]:
# ModuleNotFoundError: No module named 'pandas.core.indexes.numeric' using Metaflow
# This issue is caused by the new Pandas 2.0.0 release => pip install "pandas<2.0.0"
# https://stackoverflow.com/questions/75953279/modulenotfounderror-no-module-named-pandas-core-indexes-numeric-using-metaflo
for i,path in enumerate(file_lst):
    print(i, path)
    with open(path, 'rb') as f:
        globals()['df_event_{}'.format(i)] = pickle.load(f)
        globals()['df_event_{}'.format(i)].reset_index(inplace = True, drop = True)

0 ../../../data/pre_processed_split_data_0.pkl
1 ../../../data/pre_processed_split_data_1.pkl
2 ../../../data/pre_processed_split_data_2.pkl
3 ../../../data/pre_processed_split_data_3.pkl
4 ../../../data/pre_processed_split_data_4.pkl
5 ../../../data/pre_processed_split_data_5.pkl
6 ../../../data/pre_processed_split_0tpt_data_6.pkl
7 ../../../data/pre_processed_split_data_7.pkl
8 ../../../data/pre_processed_split_data_8.pkl


## Outlier 제거
- 참고: https://stackoverflow.com/questions/35827863/remove-outliers-in-pandas-dataframe-using-percentiles

In [20]:
# Add delta and remove outlier
def add_delta(data_frame):
    for column in ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP']:
        column_pctchg = column+'_pctchg'
        data_frame[column_pctchg] = data_frame.groupby(['id_label'])[column].pct_change()
        data_frame[column_pctchg][data_frame[column_pctchg].isna()] = 0
    return data_frame

def remove_outlier(df):
    target_cols = ['P-PDG', 'P-TPT', 'T-TPT','P-MON-CKP', 'T-JUS-CKP',]
    df = df[df['id_label'].str.startswith('WELL')]
    df = add_delta(df)
    Q1 = df[target_cols].quantile(0.25)
    Q3 = df[target_cols].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[target_cols] < (Q1 - 1.5 * IQR)) | (df[target_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df

0
1
2
3
4
5
6
7
8


In [ ]:
for i in range(9):
    print(i)
    df_event_0_out = remove_outlier(df_event_0)
    df_event_1_out = remove_outlier(df_event_1)
    df_event_2_out = remove_outlier(df_event_2)
    df_event_3_out = remove_outlier(df_event_3)
    df_event_4_out = remove_outlier(df_event_4)
    df_event_5_out = remove_outlier(df_event_5)
    df_event_6_out = remove_outlier(df_event_6)
    df_event_7_out = remove_outlier(df_event_7)
    df_event_8_out = remove_outlier(df_event_8)

# Train Test split

In [21]:
# extract the list of each data type(WELL, SIMULATED, DRAWN)
instance_dict = {'WELL': [], 'SIMULATED': [], 'DRAWN': []}
for i in range(9):
  instance_lst = list(globals()['df_event_{}'.format(i)]['id_label'].unique())
  for instance in instance_lst:
    if instance.startswith('WELL'):
      instance_dict['WELL'].extend([instance])
    elif instance.startswith('SIMULATED'):
      instance_dict['SIMULATED'].extend([instance])
    elif instance.startswith('DRAWN'):
      instance_dict['DRAWN'].extend([instance])

In [22]:
# check the number of events in 'WELL' data
class_lst = []
for i in instance_dict['WELL']:
  class_lst.append(i.split('_')[-1])
Counter(class_lst)

Counter({'0': 511, '4': 344, '3': 32, '6': 6, '7': 5, '1': 4, '2': 3, '5': 3})

In [23]:
# split train and test instances
def split_train_test_instance(instance_dict, dTypes = ['WELL', 'SIMULATED', 'DRAWN'],train_ratio = 0.6, test_ratio = 0.2):
    total_len = 0
    simul_drawn_lst = []
    for dType in dTypes:
        total_len += len(instance_dict[dType])
        simul_drawn_lst += instance_dict[dType]
    well_len = len(instance_dict['WELL'])

    num_of_test = int(total_len*test_ratio)
    num_of_val = int(total_len*(1-train_ratio-test_ratio))
    num_of_train = total_len - num_of_test - num_of_val

    # extract test_lst first(Only WELL data)
    test_lst = random.sample(instance_dict['WELL'], num_of_test)
    
    # then make whole lst --> shuffle lst --> split train validation set
    rest_well_lst = [i for i in instance_dict['WELL'] if i not in test_lst]
    total_lst = rest_well_lst + simul_drawn_lst
    random.shuffle(total_lst)

    train_lst = total_lst[:num_of_train]
    val_lst = total_lst[num_of_train:]

    return train_lst, val_lst, test_lst

In [26]:
train_ratio = 0.7
test_ratio = 0.15
val_ratio = 1 - train_ratio - test_ratio
dTypes = ['WELL', 'SIMULATED', 'DRAWN']
train_lst, val_lst, test_lst = split_train_test_instance(instance_dict, dTypes = dTypes, train_ratio = train_ratio, test_ratio = test_ratio)

print(len(train_lst), len(val_lst), len(test_lst), sep = '\n')
print('total number of lst:', len(instance_dict['WELL'])+len(instance_dict['DRAWN'])+len(instance_dict['SIMULATED']))
print('total number of lst after removing duplicated:', len(set(train_lst + val_lst + test_lst)))
print('train:', sorted(set([i.split('_')[-1] for i in train_lst])), 'val:', sorted(set([i.split('_')[-1] for i in val_lst])), 'test:', sorted(set([i.split('_')[-1] for i in test_lst])), sep = '\n')

1307
1188
280
total number of lst: 1867
total number of lst after removing duplicated: 1867
train:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
val:
['0', '1', '2', '3', '4', '5', '6', '7', '8']
test:
['0', '1', '2', '3', '4', '5', '6', '7']


In [27]:
# 참고: https://rfriend.tistory.com/590
def make_empty_df(pca_n_components):
    pca_lst = ['pca_1', 'pca_2','pca_3', 'pca_4', 'pca_5']
    columns = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
               'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
               'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
               'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
               'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
               'class',]
    columns.extend(pca_lst[:pca_n_components])
    data_frame = pd.DataFrame(columns=columns)
    return data_frame


def apply_window(data_frame, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
    if pca_n_components:
        pca = PCA(n_components=pca_n_components)
    
    new_df = make_empty_df(pca_n_components)
    for n in tqdm(range(int(len(data_frame)/(window_size*(1-overlap_ratio))))):
        try:
            if int(window_size*(n+1) - window_size*overlap_ratio*n) <= len(data_frame):
                # print(window_size*n*(1-overlap_ratio),window_size*(n+1) - window_size*overlap_ratio*n)
                sliced_df = data_frame[int(window_size*n*(1-overlap_ratio)):int(window_size*(n+1) - window_size*overlap_ratio*n)]
                pca_data = pca.fit_transform(sliced_df.drop( ['id_label','class'], axis=1))
                if any(sliced_df['class'] != 0):
                    class_label =  int(sliced_df['class'].value_counts().index[sliced_df['class'].value_counts().index < 10][0]) # 기본 모드
                    
                    # class_label = 1
                else:
                    # continue
                    class_label = 0
                new_df = pd.concat([new_df,pd.DataFrame({'P-PDG':[sliced_df['P-PDG'].mean()],
                                                        'P-PDG_std':[sliced_df['P-PDG'].std()],
                                                        'P-PDG_pctchg_mean':[sliced_df['P-PDG_pctchg'].mean()],
                                                        'P-PDG_pctchg_std':[sliced_df['P-PDG_pctchg'].std()],
                                                        'P-TPT':[sliced_df['P-TPT'].mean()],
                                                        'P-TPT_std':[sliced_df['P-TPT'].std()],
                                                        'P-TPT_pctchg_mean':[sliced_df['P-TPT_pctchg'].mean()],
                                                        'P-TPT_pctchg_std':[sliced_df['P-TPT_pctchg'].std()],
                                                        'T-TPT':[sliced_df['T-TPT'].mean()],
                                                        'T-TPT_std':[sliced_df['T-TPT'].std()],
                                                        'T-TPT_pctchg_mean':[sliced_df['T-TPT_pctchg'].mean()],
                                                        'T-TPT_pctchg_std':[sliced_df['T-TPT_pctchg'].std()],
                                                        'P-MON-CKP':[sliced_df['P-MON-CKP'].mean()],
                                                        'P-MON-CKP_std':[sliced_df['P-MON-CKP'].std()],
                                                        'P-MON-CKP_pctchg_mean':[sliced_df['P-MON-CKP_pctchg'].mean()],
                                                        'P-MON-CKP_pctchg_std':[sliced_df['P-MON-CKP_pctchg'].std()],
                                                        'T-JUS-CKP':[sliced_df['T-JUS-CKP'].mean()],
                                                        'T-JUS-CKP_std':[sliced_df['T-JUS-CKP'].std()],
                                                        'T-JUS-CKP_pctchg_mean':[sliced_df['T-JUS-CKP_pctchg'].mean()],
                                                        'T-JUS-CKP_pctchg_std':[sliced_df['T-JUS-CKP_pctchg'].std()],
                                                        'class': class_label,
                                                        'pca_1':[pca_data[:,0].mean()],
                                                        'pca_2':[pca_data[:,1].mean()],
                                                        #'pca_3':[pca_data.iloc[:,2].mean()],
                                                        })])

        except:
            continue
    return new_df

def build_dataset(data_frame, train_lst, val_lst, test_lst, window_size = 5000, overlap_ratio = 0.2, pca_n_components = 2):
  gc.collect()
  train_data_frame = data_frame[data_frame['id_label'].isin(train_lst)]
  val_data_frame = data_frame[data_frame['id_label'].isin(val_lst)]
  test_data_frame = data_frame[data_frame['id_label'].isin(test_lst)]

#   train_data_frame = remove_outlier(train_data_frame)

  train_df = apply_window(train_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  val_df = apply_window(val_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  test_df = apply_window(test_data_frame, window_size = window_size, overlap_ratio = overlap_ratio)
  return train_df, val_df, test_df

In [28]:
# applying window
window_size = 1200
window_size_short = window_size
window_size_long =  window_size
pca_n_components = 2
overlap_ratio = 0.95

df_event_0_window_train, df_event_0_window_val, df_event_0_window_test = build_dataset(df_event_0_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_1_window_train, df_event_1_window_val, df_event_1_window_test = build_dataset(df_event_1_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_2_window_train, df_event_2_window_val, df_event_2_window_test = build_dataset(df_event_2_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_3_window_train, df_event_3_window_val, df_event_3_window_test = build_dataset(df_event_3_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_4_window_train, df_event_4_window_val, df_event_4_window_test = build_dataset(df_event_4_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_5_window_train, df_event_5_window_val, df_event_5_window_test = build_dataset(df_event_5_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_6_window_train, df_event_6_window_val, df_event_6_window_test = build_dataset(df_event_6_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_7_window_train, df_event_7_window_val, df_event_7_window_test = build_dataset(df_event_7_out, train_lst, val_lst, test_lst, window_size = window_size_long, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
df_event_8_window_train, df_event_8_window_val, df_event_8_window_test = build_dataset(df_event_8_out, train_lst, val_lst, test_lst, window_size = window_size_short, overlap_ratio = overlap_ratio, pca_n_components = pca_n_components)
gc.collect()


100%|██████████| 1339/1339 [00:06<00:00, 212.83it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


0

In [38]:
# Downsampling for train dataset
merged_df_train =  pd.concat([df_event_0_window_train.sample(int(len(df_event_0_window_train)*0.1)),
                              df_event_1_window_train,
                              df_event_2_window_train,
                              df_event_3_window_train.sample(int(len(df_event_3_window_train)*0.1)),
                              df_event_4_window_train.sample(int(len(df_event_4_window_train)*0.1)),
                              df_event_5_window_train,
                              df_event_6_window_train,
                              df_event_7_window_train,
                              df_event_8_window_train])

merged_df_val =  pd.concat([df_event_0_window_val,
                            df_event_1_window_val,
                            df_event_2_window_val,
                            df_event_3_window_val,
                            df_event_4_window_val,
                            df_event_5_window_val,
                            df_event_6_window_val,
                            df_event_7_window_val,
                            df_event_8_window_val])

merged_df_test =  pd.concat([df_event_0_window_test,
                             df_event_1_window_test,
                             df_event_2_window_test,
                             df_event_3_window_test,
                             df_event_4_window_test,
                             df_event_5_window_test,
                             df_event_6_window_test,
                             df_event_7_window_test,
                             df_event_8_window_test])


In [39]:
print(merged_df_train['class'].value_counts())
print(merged_df_val['class'].value_counts())
print(merged_df_test['class'].value_counts())

class
0    10070
4     2301
3      565
6      198
5       70
7       64
2       58
Name: count, dtype: int64
class
0    56869
4    22300
3     6469
6       96
7       59
5       30
2        3
Name: count, dtype: int64
class
0    31604
4     8919
3     1569
5       30
2       19
6       10
Name: count, dtype: int64


In [40]:
# P-PDG_pctchg_mean = -inf, 혹은 inf인 row 제외: train(14개), val(10개), test(13)
merged_df_train = merged_df_train[~merged_df_train['P-PDG_pctchg_std'].isna()]
merged_df_val = merged_df_val[~merged_df_val['P-PDG_pctchg_std'].isna()]
merged_df_test = merged_df_test[~merged_df_test['P-PDG_pctchg_std'].isna()]

In [41]:
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']
X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_train_pre

,P-PDG,P-PDG_std,P-PDG_pctchg_mean,P-PDG_pctchg_std,P-TPT,P-TPT_std,P-TPT_pctchg_mean,P-TPT_pctchg_std,T-TPT,T-TPT_std,...,P-MON-CKP,P-MON-CKP_std,P-MON-CKP_pctchg_mean,P-MON-CKP_pctchg_std,T-JUS-CKP,T-JUS-CKP_std,T-JUS-CKP_pctchg_mean,T-JUS-CKP_pctchg_std,pca_1,pca_2
0,0.000000e+00,0.000000e+00,0.0,0.0,8.565903e+06,61056.170215,2.078730e-05,0.000011,118.041962,0.222718,...,1.749214e+06,236860.281168,-3.093040e-05,0.008582,75.105912,0.472400,-2.244456e-06,0.000262,-3.414849e-11,8.723388e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.419693e+06,4233.729173,-1.972196e-06,0.000022,117.029977,0.006837,...,1.667997e+06,230222.740329,5.592193e-05,0.010094,74.859587,0.356903,-6.398129e-06,0.000286,-3.104409e-12,5.217347e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.748236e+06,15403.806555,2.078632e-06,0.000014,117.459051,0.047106,...,1.759294e+06,196850.107693,6.716495e-05,0.008226,75.750729,0.587198,5.073188e-07,0.000371,1.008933e-10,-5.122274e-11
0,0.000000e+00,0.000000e+00,0.0,0.0,8.828215e+06,10590.417050,-1.189768e-06,0.000018,117.624216,0.031579,...,1.759798e+06,200198.675728,-2.761304e-05,0.008616,77.141361,0.478572,-3.179340e-06,0.000351,-4.656613e-11,7.698933e-10
0,0.000000e+00,0.000000e+00,0.0,0.0,8.658093e+06,169.749977,-1.851063e-07,0.000001,117.786254,0.015714,...,1.782538e+06,258083.413997,1.933665e-05,0.011107,76.792589,0.489672,-4.050418e-07,0.000327,6.364038e-11,-2.483572e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000e+00,0.000000,118.142814,0.010842,...,1.107130e+07,4943.190623,-2.452558e-07,0.000013,69.341553,0.059850,1.089206e-06,0.000003,3.094850e+26,5.820766e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000e+00,0.000000,118.141607,0.010670,...,1.107156e+07,4883.483012,9.712145e-07,0.000013,69.345515,0.059899,4.368168e-07,0.000006,3.094850e+26,-1.940255e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000e+00,0.000000,118.140617,0.011122,...,1.107207e+07,4658.242472,5.985480e-07,0.000013,69.346796,0.059158,1.142284e-06,0.000011,3.094850e+26,1.940255e-13
0,-1.180116e+42,3.096140e+26,0.0,0.0,2.078746e+07,0.000000,0.000000e+00,0.000000,118.140404,0.011407,...,1.107242e+07,4581.638458,4.441322e-07,0.000013,69.355407,0.059855,2.365953e-06,0.000013,3.094850e+26,-1.940255e-13


In [42]:
# normalize
# Pandas automatically applies colomn-wise function in the code above.
# merged_new_df = (merged_df - merged_df.mean())/merged_df.std()
features = ['P-PDG', 'P-PDG_std','P-PDG_pctchg_mean', 'P-PDG_pctchg_std',
            'P-TPT', 'P-TPT_std', 'P-TPT_pctchg_mean', 'P-TPT_pctchg_std',
            'T-TPT', 'T-TPT_std', 'T-TPT_pctchg_mean', 'T-TPT_pctchg_std',
            'P-MON-CKP', 'P-MON-CKP_std', 'P-MON-CKP_pctchg_mean', 'P-MON-CKP_pctchg_std',
            'T-JUS-CKP', 'T-JUS-CKP_std', 'T-JUS-CKP_pctchg_mean', 'T-JUS-CKP_pctchg_std',
            'pca_1', 'pca_2']




X_train_pre, y_train = merged_df_train.loc[:,features], merged_df_train['class']
X_val_pre, y_val = merged_df_val.loc[:,features], merged_df_val['class']
X_test_pre, y_test = merged_df_test.loc[:,features], merged_df_test['class']

# 각 칼럼의 std는 정규화에서 제외
columns = ['P-PDG', 'P-PDG_pctchg_mean','P-TPT','P-TPT_pctchg_mean', 'T-TPT','T-TPT_pctchg_mean', 'P-MON-CKP','P-MON-CKP_pctchg_mean', 'T-JUS-CKP','T-JUS-CKP_pctchg_mean',
           'P-PDG_std', 'P-TPT_std', 'T-TPT_std', 'P-MON-CKP_std', 'T-JUS-CKP_std',
           'pca_1', 'pca_2']

X_train_pre.loc[:,columns] = (X_train_pre.loc[:,columns] - X_train_pre.loc[:,columns].mean())/X_train_pre.loc[:,columns].std()
X_val_pre.loc[:,columns] = (X_val_pre.loc[:,columns] - X_val_pre.loc[:,columns].mean())/X_val_pre.loc[:,columns].std()
X_test_pre.loc[:,columns] = (X_test_pre.loc[:,columns] - X_test_pre.loc[:,columns].mean())/X_test_pre.loc[:,columns].std()

X_train = X_train_pre
X_val = X_val_pre
X_test = X_test_pre

In [44]:
# training
X_train = X_train[features].values
y_train = y_train.values.astype(int)

X_val = X_val[features].values
y_val = y_val.values.astype(int)

X_test = X_test[features].values
y_test = y_test.values.astype(int)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [45]:
# options
optimizer_fn = torch.optim.AdamW
optimizer_params = dict(lr=0.0001)
batch_size = 64
patience = 10

In [46]:
print(f'{optimizer_fn.__name__}, {train_ratio}, {val_ratio: .2f}, {test_ratio}, long = {window_size_long} short = {window_size_short}, {overlap_ratio}, lr = {optimizer_params}, batch_size = {batch_size}, no_weigth_loss, patience = {patience}, val_acc = ?, test_acc = ?, no Norm, no Simul')

weight_for_class = []
total_obs = len(y_train)
# check
for key, value in sorted(Counter(y_train).items()):
    print(f'{key} is in event, {value}')
    weight_for_class.append(1 - (value/total_obs))

max_epochs = 200
weight_for_class = torch.Tensor(weight_for_class).type(torch.float32)
print(weight_for_class)

clf = TabNetClassifier(
    optimizer_fn=optimizer_fn,
    optimizer_params=optimizer_params,#1e-2
    # cat_emb_dim = [window_size, window_size, window_size, window_size, window_size]
    )  #TabNetRegressor()

result = clf.fit(
    X_train = X_train,
    y_train = y_train,
    eval_set=[(X_val, y_val)],
    eval_metric = ["accuracy", ],
    loss_fn = nn.CrossEntropyLoss(weight_for_class),
    # loss_fn = nn.CrossEntropyLoss(),
    batch_size = batch_size,
    max_epochs = max_epochs,
    drop_last = True,
    patience = patience
)
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = 0.96924, test_acc = 0.954, no Norm, no Simul
# AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.001}, batch_size = 128, no_weigth_loss, patience = 15, val_acc = .883, test_acc = .957, no Norm, no Simul

AdamW, 0.7,  0.15, 0.15, long = 1200 short = 1200, 0.95, lr = {'lr': 0.0001}, batch_size = 64, no_weigth_loss, patience = 10, val_acc = ?, test_acc = ?, no Norm, no Simul
0 is in event, 10070
2 is in event, 58
3 is in event, 565
4 is in event, 2301
5 is in event, 70
6 is in event, 198
7 is in event, 64
tensor([0.2443, 0.9956, 0.9576, 0.8273, 0.9947, 0.9851, 0.9952])
epoch 0  | loss: 2.69135 | val_0_accuracy: 0.27839 |  0:00:06s
epoch 1  | loss: 2.5573  | val_0_accuracy: 0.37937 |  0:00:11s
epoch 2  | loss: 2.36449 | val_0_accuracy: 0.47437 |  0:00:17s
epoch 3  | loss: 2.24265 | val_0_accuracy: 0.48118 |  0:00:23s
epoch 4  | loss: 2.11222 | val_0_accuracy: 0.39928 |  0:00:29s
epoch 5  | loss: 2.04116 | val_0_accuracy: 0.3405  |  0:00:35s
epoch 6  | loss: 1.94527 | val_0_accuracy: 0.33543 |  0:00:41s
epoch 7  | loss: 1.84265 | val_0_accuracy: 0.34008 |  0:00:47s
epoch 8  | loss: 1.78735 | val_0_accuracy: 0.35988 |  0:00:52s
epoch 9  | loss: 1.68423 | val_0_accuracy: 0.35684 |  0:00:58s
e

In [37]:
preds = clf.predict(X_test)
print(accuracy_score(preds, y_test))
print(classification_report(y_test, preds))

0.9618039904154113
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     31604
           2       0.00      0.00      0.00        19
           3       0.97      0.96      0.96      1569
           4       0.86      1.00      0.92      8919
           5       0.00      0.00      0.00        30
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         0

    accuracy                           0.96     42151
   macro avg       0.40      0.42      0.41     42151
weighted avg       0.97      0.96      0.96     42151



In [ ]:
# saving_path_name = './tabnet_test_acc_9904'
# clf.save_model(saving_path_name)